# Visa Requirements Agent Demo - Walkthrough

This notebook demonstrates the multi-agent system for automating visa requirements capture.

## Overview

The system consists of 5 specialized agents:
1. **PolicyEvaluator** - Analyzes policy documents
2. **RequirementsCapture** - Extracts requirements
3. **QuestionGenerator** - Generates application questions
4. **ValidationAgent** - Validates outputs
5. **ConsolidationAgent** - Synthesizes final specification

## Setup

In [ ]:
import sys
from pathlib import Path
import json
from dotenv import load_dotenv

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root / '.env')

print(f"Project root: {project_root}")

## Import Required Modules

In [ ]:
from src.orchestrator.workflow_orchestrator import WorkflowOrchestrator
from src.agents import (
    PolicyEvaluatorAgent,
    RequirementsCaptureAgent,
    QuestionGeneratorAgent,
    ValidationAgent,
    ConsolidationAgent
)
from src.utils.document_parser import DocumentParser
from src.utils.output_formatter import OutputFormatter

print("✅ Modules imported successfully")

## Load Policy Document

In [ ]:
# Load the Parent Boost Visitor Visa policy
policy_path = project_root / 'data' / 'input' / 'parent_boost_policy.txt'
policy_text = DocumentParser.load_document(str(policy_path))

print(f"Policy document loaded: {len(policy_text)} characters")
print("\nFirst 500 characters:")
print(policy_text[:500])

## Method 1: Run Complete Workflow

This is the simplest approach - run all agents in sequence.

In [ ]:
# Initialize orchestrator
orchestrator = WorkflowOrchestrator()

print("✅ Orchestrator initialized")
print(f"Agents: {list(orchestrator.agents.keys())}")

In [ ]:
# Run complete workflow
print("🚀 Running complete workflow...\n")

results = orchestrator.run_workflow(str(policy_path))

print("\n✅ Workflow completed!")
print(f"Status: {results['status']}")
print(f"Duration: {results['duration_seconds']:.2f}s")
print(f"Stages: {len(results['stages'])}")

## Examine Results

### Stage 1: Policy Analysis

In [ ]:
policy_structure = results['outputs'].get('policy_structure', {})

print("📋 POLICY STRUCTURE")
print("=" * 80)
print(json.dumps(policy_structure, indent=2))

In [ ]:
eligibility_rules = results['outputs'].get('eligibility_rules', {})

print("📋 ELIGIBILITY RULES")
print("=" * 80)
print(json.dumps(eligibility_rules, indent=2))

### Stage 2: Requirements Capture

In [ ]:
functional_requirements = results['outputs'].get('functional_requirements', [])

print(f"📝 FUNCTIONAL REQUIREMENTS ({len(functional_requirements)} total)")
print("=" * 80)

for req in functional_requirements[:5]:  # Show first 5
    print(f"\n{req.get('requirement_id', 'N/A')}: {req.get('description', 'N/A')}")
    print(f"  Priority: {req.get('priority', 'N/A')}")
    print(f"  Policy Ref: {req.get('policy_reference', 'N/A')}")

In [ ]:
data_requirements = results['outputs'].get('data_requirements', [])

print(f"📊 DATA REQUIREMENTS ({len(data_requirements)} total)")
print("=" * 80)

for req in data_requirements[:5]:  # Show first 5
    print(f"\n{req.get('requirement_id', 'N/A')}: {req.get('field_name', 'N/A')}")
    print(f"  Type: {req.get('data_type', 'N/A')}")
    print(f"  Required: {req.get('required', False)}")
    print(f"  Description: {req.get('description', 'N/A')}")

In [ ]:
business_rules = results['outputs'].get('business_rules', [])

print(f"⚖️ BUSINESS RULES ({len(business_rules)} total)")
print("=" * 80)

for rule in business_rules[:5]:  # Show first 5
    print(f"\n{rule.get('rule_id', 'N/A')}: {rule.get('description', 'N/A')}")
    print(f"  Type: {rule.get('rule_type', 'N/A')}")
    print(f"  Policy Ref: {rule.get('policy_reference', 'N/A')}")

### Stage 3: Question Generation

In [ ]:
questions = results['outputs'].get('application_questions', [])

print(f"❓ APPLICATION QUESTIONS ({len(questions)} total)")
print("=" * 80)

# Group by section
sections = {}
for q in questions:
    section = q.get('section', 'General')
    if section not in sections:
        sections[section] = []
    sections[section].append(q)

for section, section_questions in sections.items():
    print(f"\n{section} ({len(section_questions)} questions)")
    print("-" * 80)
    
    for q in section_questions[:3]:  # Show first 3 per section
        print(f"\n  {q.get('question_id', 'N/A')}: {q.get('question_text', 'N/A')}")
        print(f"    Type: {q.get('input_type', 'N/A')} | Required: {q.get('required', False)}")
        if q.get('help_text'):
            print(f"    Help: {q['help_text'][:100]}...")

### Stage 4: Validation

In [ ]:
validation_report = results['outputs'].get('validation_report', {})

print("✅ VALIDATION REPORT")
print("=" * 80)
print(f"\nOverall Score: {validation_report.get('overall_score', 0):.1f}%")

req_val = validation_report.get('requirement_validation', {})
print(f"\nRequirement Validation:")
print(f"  Total: {req_val.get('total_requirements', 0)}")
print(f"  Valid: {req_val.get('valid_requirements', 0)}")
print(f"  Invalid: {req_val.get('invalid_requirements', 0)}")
print(f"  Rate: {req_val.get('validation_rate', 0):.1f}%")

q_val = validation_report.get('question_validation', {})
print(f"\nQuestion Validation:")
print(f"  Total: {q_val.get('total_questions', 0)}")
print(f"  Valid: {q_val.get('valid_questions', 0)}")
print(f"  Invalid: {q_val.get('invalid_questions', 0)}")
print(f"  Rate: {q_val.get('validation_rate', 0):.1f}%")

In [ ]:
gap_analysis = results['outputs'].get('gap_analysis', {})

print("🔍 GAP ANALYSIS")
print("=" * 80)

for gap_type, gaps in gap_analysis.items():
    if isinstance(gaps, list) and gaps:
        print(f"\n{gap_type.replace('_', ' ').title()}:")
        for gap in gaps[:3]:  # Show first 3
            if isinstance(gap, dict):
                print(f"  - {gap.get('description', gap)}")
            else:
                print(f"  - {gap}")

In [ ]:
recommendations = results['outputs'].get('recommendations', [])

print("💡 RECOMMENDATIONS")
print("=" * 80)

for rec in recommendations:
    priority = rec.get('priority', 'medium').upper()
    print(f"\n[{priority}] {rec.get('category', 'N/A').upper()}")
    print(f"  {rec.get('description', 'N/A')}")
    print(f"  Action: {rec.get('action', 'N/A')}")

### Stage 5: Consolidation

In [ ]:
summary_stats = results['outputs'].get('summary_statistics', {})

print("📊 SUMMARY STATISTICS")
print("=" * 80)
print(json.dumps(summary_stats, indent=2))

In [ ]:
traceability_matrix = results['outputs'].get('traceability_matrix', [])

print(f"🔗 TRACEABILITY MATRIX ({len(traceability_matrix)} mappings)")
print("=" * 80)

# Show first 5 mappings
for mapping in traceability_matrix[:5]:
    print(f"\nPolicy: {mapping.get('policy_reference', 'N/A')}")
    print(f"  Requirement: {mapping.get('requirement_id', 'N/A')}")
    print(f"  Type: {mapping.get('requirement_type', 'N/A')}")
    print(f"  Questions: {', '.join(mapping.get('related_questions', []))}")
    print(f"  Coverage: {mapping.get('coverage', 'N/A')}")

## Method 2: Run Individual Agents

You can also run agents individually for more control.

In [ ]:
# Example: Run only the Policy Evaluator
import yaml

# Load config
config_path = project_root / 'config' / 'agent_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

llm_config = config.get('llm', {})
agent_config = config.get('agents', {}).get('policy_evaluator', {})

# Initialize agent
policy_agent = PolicyEvaluatorAgent(
    name='PolicyEvaluator',
    config={**llm_config, **agent_config}
)

# Run agent
policy_results = policy_agent.execute({
    'policy_document_path': str(policy_path)
})

print("✅ Policy Evaluator completed")
print(f"Outputs: {list(policy_results.keys())}")

## Export Results

In [ ]:
# Save results to file
output_path = project_root / 'data' / 'output' / 'notebook_results.json'
OutputFormatter.save_json(results, str(output_path))

print(f"✅ Results saved to: {output_path}")

## Conclusion

This notebook demonstrated:

1. ✅ Loading and parsing policy documents
2. ✅ Running the complete multi-agent workflow
3. ✅ Examining outputs from each stage
4. ✅ Analyzing validation results and gaps
5. ✅ Reviewing consolidated specifications

### Key Metrics

- **Speed**: Automated process completed in minutes vs. weeks manually
- **Coverage**: Comprehensive extraction of requirements from policy
- **Traceability**: Complete mapping from policy → requirements → questions
- **Quality**: Validation scores and gap analysis ensure completeness

### Next Steps

1. Review validation recommendations
2. Address identified gaps
3. Refine questions based on feedback
4. Generate implementation artifacts
5. Deploy to production system